## 设置中文字体

In [1]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("font",family='SimHei') # 中国語フォントの設定
plt.rcParams['axes.unicode_minus']=False  # マイナス記号を正常に表示する

## 导入工具包

In [2]:
import time
import os

import numpy as np
from tqdm import tqdm

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

## 检查GPU

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')#パソコンにGPUがあるかどうかをチェックする
print('device', device)

device cpu


## 画像前処理

In [4]:
#画像前処理
from torchvision import transforms

#trainデータセット画像前処理：
#1.crop a random portion of image and resize it to a given size
#2.horizontally flip the given image randomly with a given probability
#3.convert the array into a Tensor
#4.the data is transformed to [- 1,1].
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])

# validデータセット画像前処理
#1.resize the input image to the given size.
#2.crops the given image at the center.
#3.convert the array into a Tensor
#4.the data is transformed to [- 1,1]
valid_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

## 载入图像分类数据集

In [5]:
# データセットフォルダのパス
dataset_dir = 'data_split'

In [6]:
train_path = os.path.join(dataset_dir, 'train')
valid_path = os.path.join(dataset_dir, 'val')
print('训练集路径', train_path)
print('测试集路径', valid_path)

训练集路径 data_split/train
测试集路径 data_split/val


In [7]:
from torchvision import datasets

# input trainデータセット
train_dataset = datasets.ImageFolder(train_path, train_transform)

# input validデータセット
valid_dataset = datasets.ImageFolder(valid_path, valid_transform)

In [8]:
print('训练集图像数量', len(train_dataset))
print('类别个数', len(train_dataset.classes))
print('各类别名称', train_dataset.classes)

训练集图像数量 1008
类别个数 4
各类别名称 ['其他垃圾', '厨余垃圾', '可回收垃圾', '有害垃圾']


In [9]:
print('测试集图像数量', len(valid_dataset))
print('类别个数', len(valid_dataset.classes))
print('各类别名称', valid_dataset.classes)

测试集图像数量 251
类别个数 4
各类别名称 ['其他垃圾', '厨余垃圾', '可回收垃圾', '有害垃圾']


## 类别和索引号 一一对应

In [10]:
# 各カテゴリーの名称
class_names = train_dataset.classes
n_class = len(class_names)

In [11]:
class_names

['其他垃圾', '厨余垃圾', '可回收垃圾', '有害垃圾']

In [12]:
# カテゴリとインデックスのマッピングを作成する
train_dataset.class_to_idx

{'其他垃圾': 0, '厨余垃圾': 1, '可回收垃圾': 2, '有害垃圾': 3}

In [13]:
idx_to_labels = {y:x for x,y in train_dataset.class_to_idx.items()}

In [14]:
idx_to_labels

{0: '其他垃圾', 1: '厨余垃圾', 2: '可回收垃圾', 3: '有害垃圾'}

In [15]:
# ローカルのnpyファイルとして保存する
np.save('idx_to_labels.npy', idx_to_labels)
np.save('labels_to_idx.npy', train_dataset.class_to_idx)

## 定义数据加载器DataLoader

In [16]:
from torch.utils.data import DataLoader

In [17]:
BATCH_SIZE = 32

# trainデータセット用のデータローダーを作成する
train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=4
                         )

# validデータセット用のデータローダーを作成する
valid_loader = DataLoader(valid_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=4
                        )

## 导入训练需使用的工具包

In [18]:
from torchvision import models
import torch.optim as optim

## 只微调训练模型最后一层

In [19]:
model = models.resnet18(pretrained=True) # resnet18モデルの読み込み，最後のレイヤを微調整する

model.fc = nn.Linear(model.fc.in_features, n_class)

optimizer = optim.Adam(model.fc.parameters())

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/featurize/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [20]:
model = model.to(device)

# cross-entropy loss function
criterion = nn.CrossEntropyLoss() 

#  Epoch
EPOCHS = 20

## 训练

In [21]:
# 訓練
for epoch in tqdm(range(EPOCHS)):

    model.train()

    for images, labels in train_loader:  
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels) # 現在のバッチの各サンプルについて、 cross-entropy loss functionを計算する
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

100%|██████████| 20/20 [21:51<00:00, 65.58s/it]


## 在测试集上初步测试

In [22]:
#validデータセットで初期テストを行う
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm(valid_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum()

    print('测试集上的准确率为 {:.3f} %'.format(100 * correct / total))

100%|██████████| 8/8 [00:06<00:00,  1.25it/s]

测试集上的准确率为 91.633 %


## 保存模型

In [23]:
torch.save(model, 'net.pth')